In [24]:
from sentence_transformers import SentenceTransformer,util
import torch

In [26]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\bhara\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bhara\.cache\huggingface\hub\models--sentence-transformers--multi-qa-MiniLM-L6-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
import requests

In [28]:
response = requests.get('https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/refs/heads/master/text-dataset-for-machine-learning/sbert-corpus.txt')
corpus = response.text.split('\r\n')

response = requests.get('https://raw.githubusercontent.com/laxmimerit/machine-learning-dataset/refs/heads/master/text-dataset-for-machine-learning/sbert-queries.txt')
queries = response.text.split('\r\n')

In [31]:
print(corpus)

['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.', 'The girl is carrying a baby.', 'The baby is carried by the woman', 'A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.', 'A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']


In [32]:
print(queries)


['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']


In [33]:
corpus_embed = model.encode(corpus , convert_to_tensor=True)

In [34]:
queries_embed = model.encode(queries , convert_to_tensor=True)

In [35]:
corpus_embeb_norm = util.normalize_embeddings(corpus_embed)

In [39]:
queries_embeb_norm = util.normalize_embeddings(queries_embed)

In [43]:
hits = util.semantic_search(queries_embeb_norm, corpus_embeb_norm, score_function=util.dot_score,top_k=3)

In [44]:
hits

[[{'corpus_id': 2, 'score': 1.0},
  {'corpus_id': 0, 'score': 0.8384665846824646},
  {'corpus_id': 1, 'score': 0.7468276023864746}],
 [{'corpus_id': 8, 'score': 0.9999999403953552},
  {'corpus_id': 7, 'score': 0.7612735033035278},
  {'corpus_id': 3, 'score': 0.3815288543701172}],
 [{'corpus_id': 10, 'score': 1.000000238418579},
  {'corpus_id': 9, 'score': 0.8703994750976562},
  {'corpus_id': 6, 'score': 0.37411704659461975}]]

In [52]:
for query, hit in zip(queries, hits):
    for q_hit in hit:
        id = q_hit['corpus_id']
        score = q_hit['score']
        print(query,' <> ' , corpus[id]  , ' --> ' ,score)

A man is eating pasta.  <>  A man is eating pasta.  -->  1.0
A man is eating pasta.  <>  A man is eating food.  -->  0.8384665846824646
A man is eating pasta.  <>  A man is eating a piece of bread.  -->  0.7468276023864746
Someone in a gorilla costume is playing a set of drums.  <>  Someone in a gorilla costume is playing a set of drums.  -->  0.9999999403953552
Someone in a gorilla costume is playing a set of drums.  <>  A monkey is playing drums.  -->  0.7612735033035278
Someone in a gorilla costume is playing a set of drums.  <>  The girl is carrying a baby.  -->  0.3815288543701172
A cheetah chases prey on across a field.  <>  A cheetah chases prey on across a field.  -->  1.000000238418579
A cheetah chases prey on across a field.  <>  A cheetah is running behind its prey.  -->  0.8703994750976562
A cheetah chases prey on across a field.  <>  A man is riding a white horse on an enclosed ground.  -->  0.37411704659461975
